## Connect to a database

In order to get data from a database with pandas, you first need to be able to connect to one. In this exercise, you'll practice creating a database engine to manage connections to a database, data.db. To do this, you'll use sqlalchemy's create_engine() function.

create_engine() needs a string URL to the database. For SQLite databases, that string consists of "sqlite:///", then the database file name.


---


Instructions:


<li> Import the create_engine() function from sqlalchemy.</li>

In [ ]:
# Import sqlalchemy's create_engine() function
from sqlalchemy import create_engine

<li> Use create_engine() to make a database engine for data.db.</li>


<li>Run the last line of code to show the names of the tables in the database.</li>

In [ ]:
# Import sqlalchemy's create_engine() function
from sqlalchemy import create_engine

# Create the database engine
engine = create_engine("sqlite:///data.db")

# View the tables in the database
print(engine.table_names())

## Load entire tables

In the last exercise, you saw that data.db has two tables. weather has historical weather data for New York City. hpd311calls is a subset of call records made to the city's 311 help line about housing issues.

In this exercise, you'll use the read_sql() function in pandas to load both tables. read_sql() accepts a string of either a SQL query to run, or a table to load. It also needs a way to connect to the database, like the engine in the provided code.


---


Instructions:

<li>Use read_sql() to load the hpd311calls table by name, without any SQL.</li>

In [ ]:
# Load libraries
import pandas as pd
from sqlalchemy import create_engine

# Create the database engine
engine = create_engine('sqlite:///data.db')

# Load hpd311calls without any SQL
hpd_calls = pd.read_sql("hpd311calls", engine)

# View the first few rows of data
print(hpd_calls.head())

<li> Use read_sql() and a SELECT * ... SQL query to load the entire weather table.</li>

In [ ]:
# Create the database engine
engine = create_engine("sqlite:///data.db")

# Create a SQL query to load the entire weather table
query = """
  SELECT *
  FROM weather;
"""

# Load weather with the SQL query
weather = pd.read_sql(query, engine)

# View the first few rows of data
print(weather.head())

## Selecting columns with SQL

Datasets can contain columns that are not required for an analysis, like the weather table in data.db does. Some, such as elevation, are redundant, since all observations occurred at the same place, while others contain variables we are not interested in. After making a database engine, you'll write a query to SELECT only the date and temperature columns, and pass both to read_sql() to make a dataframe of high and low temperature readings.

pandas has been loaded as pd, and create_engine() has been imported from sqlalchemy.

Note: The SQL checker is quite picky about column positions and expects fields to be selected in the specified order.


---


Instructions:


<li>Create a database engine for data.db.</li>


<li>Write a SQL query that SELECTs the date, tmax, and tmin columns from the weather table.</li>


<li>Make a dataframe by passing the query and engine to read_sql() and assign the resulting dataframe to temperatures.</li>

In [ ]:
# Create database engine for data.db
engine = create_engine("sqlite:///data.db")

# Write query to get date, tmax, and tmin from weather
query = """
SELECT date, 
       tmax, 
       tmin
  FROM weather;
"""

# Make a dataframe by passing query and engine to read_sql()
temperatures = pd.read_sql(query, engine)

# View the resulting dataframe
print(temperatures)

## Selecting rows

SQL WHERE clauses return records whose values meet the given criteria. Passing such a query to read_sql() results in a dataframe loaded with only records we are interested in, so there is less filtering to do later on.

The hpd311calls table in data.db has data on calls about various housing issues, from maintenance problems to information requests. In this exercise, you'll use SQL to focus on calls about safety.

pandas has been loaded as pd, and a database engine, engine, has been created for data.db.


---


Instructions:


<li>Create a query that selects all columns of records in hpd311calls that have 'SAFETY' as their complaint_type.</li>


<li>Use read_sql() to query the database and assign the result to the variable safety_calls.</li>


<li>Run the last section of code to create a graph of safety call counts in each borough.</li>

In [ ]:
# Create query to get hpd311calls records about safety
query = """
SELECT *
FROM hpd311calls
WHERE complaint_type = 'SAFETY';
"""

# Query the database and assign result to safety_calls
safety_calls = pd.read_sql(query, engine)

# Graph the number of safety calls by borough
call_counts = safety_calls.groupby('borough').unique_key.count()
call_counts.plot.barh()
plt.show()

## Filtering on multiple conditions

So far, you've selectively imported records that met a single condition, but it's also common to filter datasets on multiple criteria. In this exercise, you'll do just that.

The weather table contains daily high and low temperatures and precipitation amounts for New York City. Let's focus on inclement weather, where there was either an inch or more of snow or the high was at or below freezing (32° Fahrenheit). To do this, you'll need to build a query that uses the OR operator to look at values in both columns.

pandas is loaded as pd, and a database engine, engine, has been created.


Instructions:


---


<li>Create a query that selects records in weather where tmax is less than or equal to 32 degrees OR snow is greater than or equal to 1 inch.</li>


<li>Use read_sql() to query the database and assign the result to the variable wintry_days.</li>


<li>View summary statistics with the describe() method to make sure all records in the dataframe meet the given criteria.</li>

In [ ]:
# Create query for records with max temps <= 32 or snow >= 1
query = """
  SELECT *
  FROM weather
  WHERE tmax <= 32 OR snow >= 1;
"""

# Query database and assign result to wintry_days
wintry_days = pd.read_sql(query, engine)

# View summary stats about the temperatures
print(wintry_days.describe())

## Getting distinct values

Sometimes an analysis doesn't need every record, but rather unique values in one or more columns. Duplicate values can be removed after loading data into a dataframe, but it can also be done at import with SQL's DISTINCT keyword.

Since hpd311calls contains data about housing issues, we would expect most records to have a borough listed. Let's test this assumption by querying unique complaint_type/borough combinations.

pandas has been imported as pd, and the database engine has been created as engine.

Note: The SQL checker is quite picky about column positions and expects fields to be selected in the specified order.


---


Instructions:


<li>Create a query that gets DISTINCT values for borough and complaint_type (in that order) from hpd311calls.</li>


<li>Use read_sql() to load the results of the query to a dataframe, issues_and_boros.</li>


<li>Print the dataframe to check if the assumption that all issues besides literature requests appear with boroughs listed.</li>

In [ ]:
# Create query for unique combinations of borough and complaint_type
query = """
  SELECT DISTINCT borough, complaint_type
  FROM hpd311calls;
"""

# Load results of query to a dataframe
issues_and_boros = pd.read_sql(query, engine)

# Check assumption about issues and boroughs
print(issues_and_boros)

## Counting in groups

In previous exercises, you pulled data from tables, then summarized the resulting dataframes in pandas to create graphs. By using COUNT and GROUP BY in a SQL query, we can pull those summary figures from the database directly.

The hpd311calls table has a column, complaint_type, that categorizes call records by issue, such as heating or plumbing. In order to graph call volumes by issue, you'll write a SQL query that COUNTs records by complaint type.

pandas has been imported as pd, and the database engine for data.db has been created as engine.


---


Instructions:


<li>Create a SQL query that gets the complaint_type column and counts of all records from hpd311calls, grouped by complaint_type.</li>


<li>Create a dataframe with read_sql() of call counts by issue, calls_by_issue.</li>


<li>Run the last section of code to graph the number of calls for each housing issue.</li>

In [ ]:
# Create query to get call counts by complaint_type
query = """
  SELECT complaint_type, COUNT(*)
  FROM hpd311calls
  GROUP BY complaint_type
"""

# Create dataframe of call counts by issue
calls_by_issue = pd.read_sql(query, engine)

# Graph the number of calls for each housing issue
calls_by_issue.plot.barh(x="complaint_type")
plt.show()

## Working with aggregate functions

If a table contains data with higher granularity than is needed for an analysis, it can make sense to summarize the data with SQL aggregate functions before importing it. For example, if you have data of flood event counts by month but precipitation data by day, you may decide to SUM precipitation by month.

The weather table contains daily readings for four months. In this exercise, you'll practice summarizing weather by month with the MAX, MIN, and SUM functions.

pandas has been loaded as pd, and a database engine, engine, has been created.


---


Instructions:


<li>Create a query to pass to read_sql() that will get months and the MAX value of tmax by monthfrom weather.</li>

In [ ]:
# Create a query to get month and max tmax by month
query = """
  SELECT month, MAX(tmax)
  FROM weather
  GROUP BY month
"""

# Get dataframe of monthly weather stats
weather_by_month = pd.read_sql(query, engine)

# View weather stats by month
print(weather_by_month)

<li>Modify the query to also get the MIN tmin value for each month.</li>

In [ ]:
# Create a query to get month, max tmax, and min tmin by month
query = """
  SELECT month, MAX(tmax), MIN(tmin)
  FROM weather 
  GROUP BY month;
"""

# Get dataframe of monthly weather stats
weather_by_month = pd.read_sql(query, engine)

# View weather stats by month
print(weather_by_month)

<li>Modify the query to also get the total precipitation (prcp) for each month.</li>

In [ ]:
# Create query to get temperature and precipitation by month
query = """
  SELECT month, MAX(tmax), MIN(tmin), SUM(prcp)
  FROM weather 
  GROUP BY month;
"""

# Get dataframe of monthly weather stats
weather_by_month = pd.read_sql(query, engine)

# View weather stats by month
print(weather_by_month)

## Joining tables

Tables in relational databases usually have key columns of unique record identifiers. This lets us build pipelines that combine tables using SQL's JOIN operation, instead of having to combine data after importing it.

The records in hpd311calls often concern issues, like leaks or heating problems, that are exacerbated by weather conditions. In this exercise, you'll join weather data to call records along their common date columns to get everything in one dataframe. You can assume these columns have the same data type.

pandas is loaded as pd, and the database engine, engine, has been created.

Note: The SQL checker is picky about join table order -- it expects specific tables on the left and the right.


---


Instructions:


<li>Complete the query to join weather to hpd311calls by their date and created_date columns, respectively.</li>


<li>Query the database and assign the resulting dataframe to calls_with_weather.</li>


<li>Print the first few rows of calls_with_weather to confirm all columns were joined.</li>

In [ ]:
# Query to join weather to call records by date columns
query = """
  SELECT * 
  FROM hpd311calls
  JOIN weather 
  ON hpd311calls.created_date = weather.date;
"""

# Create dataframe of joined tables
calls_with_weather = pd.read_sql(query, engine)

# View the dataframe to make sure all columns were joined
print(calls_with_weather.head())

## Joining and filtering

Just as you might not always want all the data in a single table, you might not want all columns and rows that result from a JOIN. In this exercise, you'll use SQL to refine a data import.

Weather exacerbates some housing problems more than others. Your task is to focus on water leak reports in hpd311calls and assemble a dataset that includes the day's precipitation levels from weather to see if there is any relationship between the two. The provided SQL gets all columns in hpd311calls, but you'll need to modify it to get the necessary weather column and filter rows with a WHERE clause.

pandas is loaded as pd, and the database engine, engine, has been created.


---


Instructions:


<li>Complete query to get the prcp column in weather and join weather to hpd311calls on their date and created_date columns, respectively.</li>


<li>Use read_sql() to load the results of the query into the leak_calls dataframe.</li>

In [ ]:
# Query to get hpd311calls and precipitation values
query = """
  SELECT hpd311calls.*, weather.prcp
  FROM hpd311calls
  JOIN weather
  ON hpd311calls.created_date = weather.date;"""

# Load query results into the leak_calls dataframe
leak_calls = pd.read_sql(query, engine)

# View the dataframe
print(leak_calls.head())

<li>Modify query to get only rows that have 'WATER LEAK' as their complaint_type.</li>

In [ ]:
# Query to get water leak calls and daily precipitation
query = """
SELECT hpd311calls.*, weather.prcp
  FROM hpd311calls
  JOIN weather
    ON hpd311calls.created_date = weather.date
  WHERE hpd311calls.complaint_type = 'WATER LEAK';"""

# Load query results into the leak_calls dataframe
leak_calls = pd.read_sql(query, engine)

# View the dataframe
print(leak_calls.head())

## Joining, filtering, and aggregating

In this exercise, you'll use what you've learned to assemble a dataset to investigate how the number of heating complaints to New York City's 311 line varies with temperature.

In addition to the hpd311calls table, data.db has a weather table with daily high and low temperature readings for NYC. We want to get each day's count of heat/hot water calls with temperatures joined in. This can be done in one query, which we'll build in parts.

In part one, we'll get just the data we want from hpd311calls. Then, in part two, we'll modify the query to join in weather data.

pandas has been imported as pd, and the database engine has been created as engine.


---


Instructions:


<li>Complete the query to get created_date and counts of records whose complaint_type is HEAT/HOT WATER from hpd311calls by date.</li>


<li>Create a dataframe,df, containing the results of the query.</li>

In [ ]:
# Query to get heat/hot water call counts by created_date
query = """
SELECT hpd311calls.created_date, COUNT(*)
FROM hpd311calls
WHERE hpd311calls.complaint_type = 'HEAT/HOT WATER'
GROUP BY hpd311calls.created_date;
"""

# Query database and save results as df
df = pd.read_sql(query, engine)

# View first 5 records
print(df.head())

<li>Modify the query to join tmax and tmin from the weather table. (There is only one record per date in weather, so we do not need SQL's MAX and MIN functions here.) Join the tables on created_date in hpd311calls and date in weather.</li>

In [ ]:
# Modify query to join tmax and tmin from weather by date
query = """
SELECT hpd311calls.created_date, 
	   COUNT(*), 
       weather.tmax,
       weather.tmin
  FROM hpd311calls 
       JOIN weather
       ON hpd311calls.created_date = weather.date
 WHERE hpd311calls.complaint_type = 'HEAT/HOT WATER' 
 GROUP BY hpd311calls.created_date;
 """

# Query database and save results as df
df = pd.read_sql(query, engine)

# View first 5 records
print(df.head())